In [1]:
# Check if running on Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running on Google Colab")
except:
    IN_COLAB = False
    print("📝 Running on local Jupyter")

# Check GPU availability
import tensorflow as tf
print(f"\nTensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

if tf.config.list_physical_devices('GPU'):
    print("🚀 GPU detected! Training will be accelerated.")
else:
    print("⚠️ No GPU detected. Training will be slower on CPU.")
    if IN_COLAB:
        print("💡 Enable GPU: Runtime > Change runtime type > Hardware accelerator > GPU")

✅ Running on Google Colab

TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
🚀 GPU detected! Training will be accelerated.


In [2]:
import os, shutil

os.chdir('/content')
if os.path.exists('EstraNet'):
    shutil.rmtree('EstraNet')  # Remove nested mess
    
!git clone https://github.com/loshithan/EstraNet.git
os.chdir('EstraNet')
print(f"✅ Clean! Directory: {os.getcwd()}")

Cloning into 'EstraNet'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 95 (delta 43), reused 88 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 2.92 MiB | 8.36 MiB/s, done.
Resolving deltas: 100% (43/43), done.
✅ Clean! Directory: /content/EstraNet


In [3]:
import os
import gdown

# Create data directory
os.makedirs('data', exist_ok=True)

# ASCADf dataset configuration
file_id = "1WNajWT0qFbpqPJiuePS_HeXxsCvUHI5M"
DATASET_PATH = "data/ASCAD.h5"

if not os.path.exists(DATASET_PATH):
    print("📥 Downloading ASCADf dataset from Google Drive...")
    print("   This may take a few minutes (~1.5 GB)\n")
    
    # Download using gdown
    gdown.download(f"https://drive.google.com/uc?id={file_id}", DATASET_PATH, quiet=False)
    
    print("\n✅ Dataset downloaded successfully!")
else:
    print("✅ Dataset already exists")

# Verify dataset
import h5py
with h5py.File(DATASET_PATH, 'r') as f:
    print(f"\n📊 Dataset info:")
    print(f"  Keys: {list(f.keys())}")
    if 'Profiling_traces' in f:
        print(f"  Profiling traces shape: {f['Profiling_traces/traces'].shape}")
    if 'Attack_traces' in f:
        print(f"  Attack traces shape: {f['Attack_traces/traces'].shape}")

📥 Downloading ASCADf dataset from Google Drive...
   This may take a few minutes (~1.5 GB)



Downloading...
From: https://drive.google.com/uc?id=1WNajWT0qFbpqPJiuePS_HeXxsCvUHI5M
To: /content/EstraNet/data/ASCAD.h5
100%|██████████| 46.6M/46.6M [00:00<00:00, 73.8MB/s]


✅ Dataset downloaded successfully!

📊 Dataset info:
  Keys: ['Attack_traces', 'Profiling_traces']
  Profiling traces shape: (50000, 700)
  Attack traces shape: (10000, 700)


In [ ]:
# Install required dependencies
print("📦 Installing dependencies...\n")
# %pip install -q absl-py==2.3.1 numpy==1.24.3 scipy==1.10.1 h5py==3.11.0

# Install gdown for downloading from Google Drive
%pip install -q gdown

# Note: Using TensorFlow version pre-installed in Colab (2.16+ / 2.19+)
# The compatibility fixes in Section 3 work with all TensorFlow 2.13+ versions
print("\n✅ All dependencies installed!")
print(f"Using TensorFlow {tf.__version__} (pre-installed)")

In [ ]:
# ============================================================================
# TRAIN GNN MODEL IN COLAB
# ============================================================================
# Paste this into a new Colab cell

print("🔷 Training GNN Model")
print("="*70)

# Configuration
CONFIG = {
    'checkpoint_dir': '/content/drive/MyDrive/EstraNet/checkpoints_gnn',
    'result_path': 'results/gnn',
    'train_steps': 50000,
    'save_steps': 5000,
    'train_batch_size': 256,
    'eval_batch_size': 32,
    'learning_rate': 0.00025,
    'model_type': 'gnn',  # KEY: Use GNN
}

import os
os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)
os.makedirs(CONFIG['result_path'], exist_ok=True)

# Build training command
train_cmd = f"""
python train_trans.py \\
    --data_path=data/ASCAD.h5 \\
    --checkpoint_dir={CONFIG['checkpoint_dir']} \\
    --model_type={CONFIG['model_type']} \\
    --dataset=ASCAD \\
    --input_length=10000 \\
    --eval_batch_size={CONFIG['eval_batch_size']} \\
    --n_layer=2 \\
    --d_model=128 \\
    --d_inner=256 \\
    --n_head_softmax=8 \\
    --d_head_softmax=16 \\
    --dropout=0.05 \\
    --conv_kernel_size=3 \\
    --n_conv_layer=2 \\
    --pool_size=20 \\
    --beta_hat_2=150 \\
    --model_normalization=preLC \\
    --softmax_attn=True \\
    --do_train=True \\
    --learning_rate={CONFIG['learning_rate']} \\
    --clip=0.25 \\
    --min_lr_ratio=0.004 \\
    --warmup_steps=0 \\
    --train_batch_size={CONFIG['train_batch_size']} \\
    --train_steps={CONFIG['train_steps']} \\
    --iterations=500 \\
    --save_steps={CONFIG['save_steps']} \\
    --result_path={CONFIG['result_path']}
"""

print("Starting GNN training...")
print(f"Model: GNN (211,876 parameters - 51% less than Transformer)")
print(f"Checkpoints: {CONFIG['checkpoint_dir']}")
print(f"Training steps: {CONFIG['train_steps']:,}\n")

!{train_cmd}


In [ ]:
# ============================================================================
# BACKUP GNN CHECKPOINTS TO GOOGLE DRIVE
# ============================================================================
# Paste this into a Colab cell after training

from google.colab import drive
import os
import shutil
import glob
import datetime

print("Mounting Google Drive...")
drive.mount('/content/drive')

# Create backup folder with timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
backup_dir = f"/content/drive/MyDrive/EstraNet_GNN_Backup_{timestamp}"
os.makedirs(backup_dir, exist_ok=True)
print(f"✅ Created backup folder: {backup_dir}")

# The checkpoints are already in Drive, but let's also backup locally trained files
print("\n📂 Backing up GNN checkpoint files...")

# Copy checkpoint files from local directory (if any were saved locally)
local_checkpoint_files = glob.glob("trans_long*") + glob.glob("checkpoint") + glob.glob("loss.pkl")
if local_checkpoint_files:
    print(f"Found {len(local_checkpoint_files)} local checkpoint files")
    for file in local_checkpoint_files:
        try:
            shutil.copy2(file, backup_dir)
            print(f"  ✅ {file}")
        except Exception as e:
            print(f"  ❌ {file}: {e}")
else:
    print("  No local checkpoints (they're in Drive already)")

# Copy GNN-specific code files
print("\n📄 Backing up GNN code files...")
code_files = [
    'gnn_layers.py',
    'gnn_estranet.py', 
    'train_trans.py',
    'transformer.py',
    'fast_attention.py'
]

for file in code_files:
    if os.path.exists(file):
        try:
            shutil.copy2(file, backup_dir)
            print(f"  ✅ {file}")
        except:
            print(f"  ⚠️  {file} not found")

# Copy results folder
print("\n📊 Backing up results...")
if os.path.exists("results"):
    try:
        shutil.copytree("results", f"{backup_dir}/results")
        print("  ✅ results/")
    except Exception as e:
        print(f"  ❌ Failed: {e}")

# Verify Drive checkpoints exist
print("\n🔍 Verifying GNN checkpoints in Drive...")
drive_checkpoint_dir = "/content/drive/MyDrive/EstraNet/checkpoints_gnn"
if os.path.exists(drive_checkpoint_dir):
    drive_files = os.listdir(drive_checkpoint_dir)
    checkpoint_count = len([f for f in drive_files if 'trans_long' in f or 'checkpoint' in f])
    print(f"  ✅ Found {checkpoint_count} files in {drive_checkpoint_dir}")
else:
    print(f"  ⚠️  Drive checkpoint directory not found: {drive_checkpoint_dir}")

print("\n" + "="*70)
print("🎉 BACKUP COMPLETE!")
print(f"📁 Backup location: {backup_dir}")
print("="*70)
